In [1]:
import pandas as pd
import random
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, matthews_corrcoef, confusion_matrix
from sklearn.preprocessing import label_binarize
import torch.optim as optim
import numpy as np

# Load the data
with open('/content/casualnet.json', 'r') as f:
    casulnet_data = json.load(f)


random_selected_samples = random.sample(casulnet_data, 1000)

# Define split sizes for an 80-20 split
train_size = int(0.8 * len(random_selected_samples))
train_data = random_selected_samples[:train_size]
validation_data = random_selected_samples[train_size:]

class CasulnetDataset(Dataset):
    def __init__(self, casulnet_data, tokenizer):
        self.casulnet_data = casulnet_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.casulnet_data)

    def __getitem__(self, idx):
        data = self.casulnet_data[idx]
        context_question = data['context']
        choices = [data[f'choice_id: {i}'] for i in range(3)]  # Assuming there are always 3 choices
        label = data['label']

        # Tokenizing context_question with each choice
        input_ids = []
        attention_masks = []
        for choice in choices:
            encoded_input = self.tokenizer(context_question, choice, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
            input_ids.append(encoded_input['input_ids'].squeeze(0))
            attention_masks.append(encoded_input['attention_mask'].squeeze(0))

        label = torch.tensor([label]*3)  # Replicate label for each choice
        return {
            'input_ids': torch.stack(input_ids),
            'attention_mask': torch.stack(attention_masks),
            'labels': label
        }

# Load tokenizer and model
# Load tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)  # Assuming 3 possible labels

casulnet_dataset = CasulnetDataset(train_data, tokenizer)
dataloader = DataLoader(casulnet_dataset, batch_size=8, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Training loop
model.train()
for epoch in range(3):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].view(-1, 512)  # Flatten the input for processing
        attention_mask = batch['attention_mask'].view(-1, 512)  # Flatten the attention mask for processing
        labels = batch['labels'].view(-1)  # Flatten the labels to match the input batch size

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

validation_dataset = CasulnetDataset(validation_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=64)

model.eval()
y_true, y_pred, y_scores = [], [], []

def compute_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(label_binarize(y_true, classes=[0, 1, 2]), y_prob, multi_class='ovr', average='weighted')
    mcc = matthews_corrcoef(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    return accuracy, f1, precision, recall, roc_auc, mcc, conf_matrix

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].view(-1, 512)
        attention_mask = batch['attention_mask'].view(-1, 512)
        labels = batch['labels'].view(-1)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        y_scores_batch = torch.softmax(outputs.logits, dim=1).cpu().numpy()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        y_scores.extend(y_scores_batch)

# Calculate metrics
y_scores = np.vstack(y_scores)  # Ensure y_scores is properly shaped for multiclass ROC-AUC calculation
accuracy, f1, precision, recall, roc_auc, mcc, conf_matrix = compute_metrics(y_true, y_pred, y_scores)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"MCC: {mcc:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 1.0788108110427856
Epoch: 0, Loss: 1.1145228147506714
Epoch: 0, Loss: 1.127295970916748
Epoch: 0, Loss: 1.0781593322753906
Epoch: 0, Loss: 1.0859676599502563
Epoch: 0, Loss: 1.157182216644287
Epoch: 0, Loss: 1.1113759279251099
Epoch: 0, Loss: 1.1237081289291382
Epoch: 0, Loss: 1.1010375022888184
Epoch: 0, Loss: 1.1155017614364624
Epoch: 0, Loss: 1.0655640363693237
Epoch: 0, Loss: 1.1623982191085815
Epoch: 0, Loss: 1.0925542116165161
Epoch: 0, Loss: 1.13943350315094
Epoch: 0, Loss: 1.1385020017623901
Epoch: 0, Loss: 1.1383973360061646
Epoch: 0, Loss: 1.0715081691741943
Epoch: 0, Loss: 1.1149139404296875
Epoch: 0, Loss: 1.098990559577942
Epoch: 0, Loss: 1.1146937608718872
Epoch: 0, Loss: 1.1060446500778198
Epoch: 0, Loss: 1.0975720882415771
Epoch: 0, Loss: 1.1199604272842407
Epoch: 0, Loss: 1.0970864295959473
Epoch: 0, Loss: 1.0817228555679321
Epoch: 0, Loss: 1.1125030517578125
Epoch: 0, Loss: 1.1200045347213745
Epoch: 0, Loss: 1.1094533205032349
Epoch: 0, Loss: 1.1157342

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
